In [1]:
"""
Задание к домашней работе
Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
1. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
2. Построить случайный лес, вывести важность признаков
3. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
4. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
5. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
"""

import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler # Стандартизация функций путем удаления среднего 
                                                  # и масштабирования до единичной дисперсии
from sklearn.ensemble import StackingRegressor
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
"""
1. Решаем задачу линейной регресии 
"""
data = pd.read_csv('train.csv')
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
# Выполним следующее:
# 1) подготовим датасет, освободив его от пропусков
# 2) обучаем методом регрессии, фиксируем результат
# 3) строим случайный лес классом RandomForestRegressor, создав холды через параметр min_samples_split (или класс K-Fold),
# фиксируем результат
# 4) делаем стекинг моделей со случайными, нами заданными гиперпараметрами, фиксируем результат
# 5) а теперь попробуем задать ему наилучшие гиперпараметры. Для этого находим их через через GridSearchCV
# 6) далее подставляем полученные значения в стекинг и снова обучаем модель, фиксируем результат 

In [4]:
# 1) подготовим датасет, освободив его от пропусков
# Считаем кол-во пропусков в каждой колонке:
nan_values = data.isnull().sum().sort_values(ascending=False) 
nan_values.head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
Id                 0
dtype: int64

In [5]:
# удалим пустые строки с записью NaN
data_ = pd.DataFrame.copy(data.drop(nan_values[nan_values>0].index,axis=1)) # axis='columns'== axis=1, а axis='index'== axis=0
print(data_)

        Id  MSSubClass MSZoning  LotArea Street LotShape LandContour  \
0        1          60       RL     8450   Pave      Reg         Lvl   
1        2          20       RL     9600   Pave      Reg         Lvl   
2        3          60       RL    11250   Pave      IR1         Lvl   
3        4          70       RL     9550   Pave      IR1         Lvl   
4        5          60       RL    14260   Pave      IR1         Lvl   
...    ...         ...      ...      ...    ...      ...         ...   
1455  1456          60       RL     7917   Pave      Reg         Lvl   
1456  1457          20       RL    13175   Pave      Reg         Lvl   
1457  1458          70       RL     9042   Pave      Reg         Lvl   
1458  1459          20       RL     9717   Pave      Reg         Lvl   
1459  1460          20       RL     9937   Pave      Reg         Lvl   

     Utilities LotConfig LandSlope  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0       AllPub    Inside       Gtl  ...             0   

In [6]:
#Находим категориальные признаки
cat_feat = list(data_.dtypes[data_.dtypes == object].index) # index - берем построчно 

In [7]:
# Получаем непрерывные признаки
num_feat = [f for f in data_ if f not in (cat_feat + ['Id', 'SalePrice'])]

In [8]:
# Объединим 2 вида признаков в одной переменной, чтобы определить ее в Х 

# df = data.reset_index()

factor_feat = pd.concat((data_[num_feat], data_[cat_feat]), axis=1) # объединяем столбцы фрейма data - категориальные
                                                                  # признаки и непрерывные

In [9]:
# Задаем переменные Х и y
X = data_[factor_feat.columns]    
y = data['SalePrice'] 

In [10]:
# Разбиваем на train/test
D_train, D_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Создаем дамми-переменные для категорий
dummy_train = pd.get_dummies(D_train[cat_feat], columns=cat_feat)  # преобразовываем категориальную переменную 
dummy_test = pd.get_dummies(D_test[cat_feat], columns=cat_feat)    # в фиктивные / индикаторные переменные

dummy_cols = list(set(dummy_train) & set(dummy_test))           # список уникальных (set) колонок категориальных признаков

dummy_train_q = dummy_train[dummy_cols]                         # обучающийся сет категориальных признаков 
dummy_test_q = dummy_test[dummy_cols]                           # тестовый сет категориальных признаков

In [12]:
# Объединяем 2 вида признаков в Х для обучающей и тестовой выборок 
X_train = pd.concat([D_train[num_feat],
                     dummy_train_q], axis=1)

X_test = pd.concat([D_test[num_feat],
                     dummy_test_q], axis=1)

In [13]:
# 2) определим модель линейной регресиии и обучим ее 
model = LinearRegression()
model.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
# Получим прогнозные значения
y_pred = model.predict(X_test)
# Проверим качество модели - зафиксируем результат 
model.score(X_test, y_test)

0.8724659227903068

In [15]:
# 3) строим случайный лес классом RandomForestRegressor, создав холды через параметр min_samples_split
clf_tree = RandomForestRegressor(max_depth=15, min_samples_leaf=20,  min_samples_split=10, random_state=5)
clf_tree.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=20,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=5, verbose=0, warm_start=False)

In [16]:
# проверим качество модели (высокое)
clf_tree.score(X_test, y_test)

0.814060062879311

In [17]:
# 4) делаем стекинг моделей со случайными, нами заданными гиперпараметрами, фиксируем результат

# Вначале стандартиззируем выборку численных признаков путем удаления среднего и масштабирования до единичной дисперсии
scaler = StandardScaler()
scaler.fit(X_train[num_feat])
X_train[num_feat] = scaler.fit_transform(X_train[num_feat])
X_test[num_feat] = scaler.fit_transform(X_test[num_feat])

In [18]:
# Займемся непосредственым обученем стекинга 
# Финальное решение в стеке обучающихся моделей принимает логистическая регрессия  

regressor = StackingRegressor(
    [
        ('lr', LinearRegression()),
        ('rfr', RandomForestRegressor(random_state=44)),
        ('knr', KNeighborsRegressor())
    ],
LinearRegression(), cv=10)


In [19]:
# обучим модель 
regressor.fit(X_train, y_train)

StackingRegressor(cv=10,
                  estimators=[('lr',
                               LinearRegression(copy_X=True, fit_intercept=True,
                                                n_jobs=None, normalize=False)),
                              ('rfr',
                               RandomForestRegressor(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
             

In [20]:
# проверим качество модели (высокое)
regressor.score(X_test, y_test)

0.8957271244547884

In [21]:
# 5) а теперь попробуем задать наилучшие гиперпараметры. Для этого находим их через через GridSearchCV

regr = KNeighborsRegressor()

grid_param = {
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [5,10,15,20,25,30],
    'p': [1,2],
    'metric': ['minkowski', 'euclidean', 'manhattan', 'chebyshev', 'wminkowski', 'seuclidean', 'mahalanobis'], 
    'n_neighbors': [1,2,3,4,5,6,7,8,9,10],
    'weights': ['uniform', 'distance']
#     'w': [0.1, 0.2, 0.3]
}



In [22]:
gd_sr = GridSearchCV(estimator=regr,
                     param_grid=grid_param,
#                      scoring='mean_absolute_error',
                     cv=10,
                     n_jobs=-1)
gd_sr

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [5, 10, 15, 20, 25, 30],
                         'metric': ['minkowski', 'euclidean', 'manhattan',
                                    'chebyshev', 'wminkowski', 'seuclidean',
                                    'mahalanobis'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
         

In [ ]:
# обучим модель 
gd_sr.fit(X_train, y_train)

In [ ]:
# выведем наилучшие параметры 
best_parameters = gd_sr.best_params_
best_parameters

In [ ]:
# 6) далее подставим полученные значения в стекинг и снова обучим модель, зафиксируем результат 